In [80]:
import pandas as pd 
import numpy as np
import copy
data = pd.read_csv("train.csv") 
data.dtypes


satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
left                       int64
promotion_last_5years      int64
sales                     object
salary                    object
dtype: object

In [81]:
temp5 = list(data.index.values)
numberOfPlus = 0
numberOfNeg = 0
for i in temp5:
    if str(data.at[i,"left"])==str(1):
        numberOfPlus += 1
    else:
        numberOfNeg += 1
print(numberOfPlus)
print(numberOfNeg)
ProPos = numberOfPlus/(numberOfPlus+numberOfNeg)
ProNeg = numberOfNeg/(numberOfPlus+numberOfNeg)
baseLineAccuracy = (ProPos*ProPos)+(ProNeg*ProNeg)
print(ProPos)
print(baseLineAccuracy)

2675
8563
0.2380316782345613
0.637254803217201


In [82]:
#ran = np.random.rand(len(data)) < 0.8
#train = data[ran]
#val = data[~ran]
train =  pd.read_csv('UsedTrainingData.csv')
val = pd.read_csv('UsedValidationData.csv')
print("length of training data is "+str(len(train)))
print("length of validation data is "+str(len(val)))
temp5 = list(val.index.values)
numberOfPlus = 0
numberOfNeg = 0

for i in temp5:
    if str(val.at[i,"left"])==str(1):
        numberOfPlus += 1
    else:
        numberOfNeg += 1
print(type(i))
print(numberOfPlus)
print(numberOfNeg)
print(numberOfPlus/(numberOfNeg+numberOfPlus))

length of training data is 8989
length of validation data is 2249
<class 'numpy.int64'>
522
1727
0.23210315695864828


In [4]:
#train = train.loc[:,['Work_accident', 'promotion_last_5years', 'sales', 'salary','left'] ] 
#val = val.loc[:,['Work_accident', 'promotion_last_5years', 'sales', 'salary','left'] ] 

In [ ]:
#numberOfNodes = 0

In [83]:
targetAttribute = 'left'
Attributes = list(train.columns)
Attributes.remove(targetAttribute)
AllAttrExpTar = Attributes[:]
print(Attributes)
examples = list(train.index.values) 
len(examples)


['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years', 'sales', 'salary']


8989

In [7]:
NumericalAttributes = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company']
CategoricalAttributes = ['Work_accident', 'promotion_last_5years', 'sales', 'salary']


In [85]:
def getImpurityOfExamples(examplesForCurNode,method):
    NumberOfPosEx = 0
    NumberOfNegEx = 0
    for j in examplesForCurNode:
        if  str(train.at[int(j),targetAttribute]) == str(1):
            NumberOfPosEx = NumberOfPosEx+1
        elif str(train.at[int(j),targetAttribute]) == str(0):
             NumberOfNegEx = NumberOfNegEx+1
    PropOfPos =  NumberOfPosEx/len(examplesForCurNode)
    PropOfNeg = NumberOfNegEx/len(examplesForCurNode)
    if method == 'Gini':
        return (2*PropOfPos*PropOfNeg)
    elif method == 'misClasifi':
        return min(PropOfPos,PropOfNeg)
    elif method == 'Entropy':
        if PropOfPos == 0:
            return (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2)))
        elif PropOfNeg == 0:
            return (-1*PropOfPos*(np.log(PropOfPos)/np.log(2)))
        else:
            return ((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))

In [87]:
def getMethodImpurity(Attribute,value,examplesForCurAttr,method):
    #print("calculating Impurity")    
    fra1 = train.loc[examplesForCurAttr,:]
    LessThanExamples = []
    MoreThanExamples = []
    if Attribute in NumericalAttributes: 
        for j in examplesForCurAttr:
            if  float(str(fra1.at[int(j),Attribute])) <= float(str(value)):
                LessThanExamples.append(int(j))
            else:
                MoreThanExamples.append(int(j))
    else:
        for j in examplesForCurAttr:
            if  str(fra1.at[int(j),Attribute]) == str(value):
                LessThanExamples.append(int(j))
            else:
                MoreThanExamples.append(int(j))
        
    if len(LessThanExamples) <=10 or len(MoreThanExamples) <= 0:    
        #print("I am here and there is a problem")
        return 999
    else:
        #if getEntropyOfExamples(LessThanExamples) ==0 or getEntropyOfExamples(MoreThanExamples) == 0:
            #print("yes i am here kartheek")        
        return ((len(LessThanExamples)*getImpurityOfExamples(LessThanExamples,method))+(len(MoreThanExamples)*getImpurityOfExamples(MoreThanExamples,method)))/len(examplesForCurAttr)
    


In [10]:
class Node(object):
    def __init__(self,examples,decisionAttributeType,decisionAttribute,decisionAttributeValue,level):
        self.examples = examples
        self.decisionAttributeType = decisionAttributeType
        self.decisionAttribute = decisionAttribute        
        self.decisionAttributeValue = decisionAttributeValue
        self.level = level
        self.leftChild = None
        self.rightChild = None
        
    def __str__(self): 
        return "Decision attribute type is "+self.decisionAttributeType+"decisionAttribute is "+ self.decisionAttribute+" decisionAttributeValue is "+ str(self.decisionAttributeValue)+" of level "+str(self.level)+" number of examples of this node "+str(len(self.examples))

In [88]:
def BestOfAttributes(examplesForCurAttr,targetAttribute,method):
    #entropyForCurrentExamples = getImpurityOfExamples(examplesForCurAttr,method)
    #global numberOfNodes
    
    fra1 = train.loc[examplesForCurAttr,CategoricalAttributes]
    dictForCategAttrUniqueValues = {}
    for i in CategoricalAttributes:
        dictForCategAttrUniqueValues[i] = list(fra1[i].sort_values().unique())
    dictForEntropyUniqueValueCateg = copy.deepcopy(dictForCategAttrUniqueValues)
    print(dictForCategAttrUniqueValues)
    print("calculating Impurity")
    for i in CategoricalAttributes:
        for j,val in enumerate(dictForCategAttrUniqueValues[i]):
            dictForEntropyUniqueValueCateg[i][j] = getMethodImpurity(i,dictForCategAttrUniqueValues[i][j],examplesForCurAttr,method)          
    EntropyValuesForCategoricalAttributes = {}
    for i in CategoricalAttributes:
        EntropyValuesForCategoricalAttributes[i]=min(dictForEntropyUniqueValueCateg[i])
    BestOfCategoricalAttributes = min(EntropyValuesForCategoricalAttributes, key=EntropyValuesForCategoricalAttributes.get)
    BestOfCategoricalAttributesValue = min(EntropyValuesForCategoricalAttributes.values())   
    print(EntropyValuesForCategoricalAttributes)
    
    fra1 = train.loc[examplesForCurAttr,NumericalAttributes]
    dictForNumAttrUniqueValues = {}
    for i in NumericalAttributes:
        dictForNumAttrUniqueValues[i] = list(fra1[i].sort_values().unique())
    dictForEntropyUniqueValueNum = copy.deepcopy(dictForNumAttrUniqueValues)
    #dictForEntropyUniqueValueNum = {}
    print(dictForNumAttrUniqueValues)
    print("calculating Impurity")
    for i in NumericalAttributes:
        for j,val in enumerate(dictForNumAttrUniqueValues[i]):
            dictForEntropyUniqueValueNum[i][j] = getMethodImpurity(i,dictForNumAttrUniqueValues[i][j],examplesForCurAttr,method)          
    EntropyValuesForNumericalAttributes = {}
    for i in NumericalAttributes:
        EntropyValuesForNumericalAttributes[i]=min(dictForEntropyUniqueValueNum[i])
    BestOfNumericalAttributes = min(EntropyValuesForNumericalAttributes, key=EntropyValuesForNumericalAttributes.get)
    BestOfNumericalAttributesValue = min(EntropyValuesForNumericalAttributes.values())
    print(EntropyValuesForNumericalAttributes)
    
    for index, item in enumerate(dictForEntropyUniqueValueNum[BestOfNumericalAttributes]):
        if item == BestOfNumericalAttributesValue:
            BestSplitValueNumerical = dictForNumAttrUniqueValues[BestOfNumericalAttributes][index]
            break
            
    for index, item in enumerate(dictForEntropyUniqueValueCateg[BestOfCategoricalAttributes]):
        if item == BestOfCategoricalAttributesValue:
            BestSplitValueCategorical = dictForCategAttrUniqueValues[BestOfCategoricalAttributes][index]
            break

    if BestOfNumericalAttributesValue < BestOfCategoricalAttributesValue:
        return BestOfNumericalAttributes,BestSplitValueNumerical
    else: 
        return BestOfCategoricalAttributes,BestSplitValueCategorical
    
    

In [89]:
def DecisionTreeBuilder(examplesForCurNode,targetAttribute,level,method):
    allExPos = True
    for i in examplesForCurNode:
        if str(train.at[int(i),targetAttribute]) == str(1):
            pass
        else:
            allExPos = False
            break
    if allExPos:        
        r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
        print(r)
        print("Type1")
        return r
    allExNeg = True
    for j in examplesForCurNode:
        if str(train.at[j,targetAttribute]) == str(0):
            pass
        else:
            allExNeg = False
            break
    if allExNeg:
        r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
        print(r)
        print("Type2")
        return r
    if getImpurityOfExamples(examplesForCurNode,method) < 0.01 or len(examplesForCurNode)<50:
        fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
        targetAttributesList = fra1[targetAttribute].values
        unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
        dictOfTaAttr = dict(zip(unique1, counts1))
        posCount = dictOfTaAttr[1]
        negCount = dictOfTaAttr[0]
        if (posCount/(posCount+negCount))>0.5:
            r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
            #numberOfNodes  = numberOfNodes + 1
        else:
            r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
            #numberOfNodes  = numberOfNodes + 1
        print(r)
        print("Type3")
        return r
    
    BestAttribute,BestAttributeValue = BestOfAttributes(examplesForCurNode,targetAttribute,method)
    
    if BestAttribute in NumericalAttributes:
        BestAttributeType = 'numerical'
    else:
        BestAttributeType = 'categorical'
    print("BestAttribute is"+BestAttribute)
    print("BestAttributeType is"+BestAttributeType)
    r = Node(examplesForCurNode,BestAttributeType,BestAttribute,BestAttributeValue,level)
    print(r)
    print("Type4")
    #numberOfNodes  = numberOfNodes + 1
    examplesForLeftChildOfCurrentNode = []
    examplesForRightChildOfCurrentNode = []        
    if BestAttributeType == 'categorical':        
        for iter1 in examplesForCurNode:
            if  str(train.at[int(iter1),str(BestAttribute)]) == str(BestAttributeValue):
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
    else:
        for iter1 in examplesForCurNode:
            if  float(str(train.at[int(iter1),str(BestAttribute)])) <= float(str(BestAttributeValue)):
                print(float(str(train.at[int(iter1),str(BestAttribute)])))
                print(float(str(BestAttributeValue)))
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
    print("length of left children is "+str(len(examplesForLeftChildOfCurrentNode)))
    print("length of right children is "+str(len(examplesForLeftChildOfCurrentNode)))  
    r.leftChild = DecisionTreeBuilder(examplesForLeftChildOfCurrentNode,targetAttribute,level+1,method)
    r.rightChild = DecisionTreeBuilder(examplesForRightChildOfCurrentNode,targetAttribute,level+1,method)
    
    return r

In [91]:
#numberOfNodes = 0
examplesForRootNode = examples[:]
AttributesForRootNode = Attributes[:]
method = 'Gini'
RootNode = DecisionTreeBuilder(examplesForRootNode,targetAttribute,0,method)

{'Work_accident': [0, 1], 'promotion_last_5years': [0, 1], 'sales': ['IT', 'RandD', 'accounting', 'hr', 'management', 'marketing', 'product_mng', 'sales', 'support', 'technical'], 'salary': ['high', 'low', 'medium']}
calculating Impurity
{'Work_accident': 0.3560035240679372, 'promotion_last_5years': 0.36277419104361847, 'sales': 0.36328827747951464, 'salary': 0.35803980274168795}
{'satisfaction_level': [0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'last_evaluation': [0.36, 0.37, 0.38, 0.39, 0.4, 0

0.09
0.46
0.39
0.46
0.32
0.46
0.22
0.46
0.41
0.46
0.17
0.46
0.4
0.46
0.45
0.46
0.16
0.46
0.41
0.46
0.1
0.46
0.1
0.46
0.42
0.46
0.45
0.46
0.1
0.46
0.42
0.46
0.1
0.46
0.4
0.46
0.36
0.46
0.1
0.46
0.44
0.46
0.11
0.46
0.44
0.46
0.11
0.46
0.42
0.46
0.17
0.46
0.1
0.46
0.23
0.46
0.37
0.46
0.16
0.46
0.28
0.46
0.41
0.46
0.42
0.46
0.45
0.46
0.36
0.46
0.43
0.46
0.43
0.46
0.29
0.46
0.32
0.46
0.11
0.46
0.45
0.46
0.41
0.46
0.41
0.46
0.37
0.46
0.09
0.46
0.11
0.46
0.11
0.46
0.37
0.46
0.4
0.46
0.2
0.46
0.4
0.46
0.31
0.46
0.44
0.46
0.23
0.46
0.41
0.46
0.18
0.46
0.11
0.46
0.44
0.46
0.19
0.46
0.19
0.46
0.32
0.46
0.15
0.46
0.43
0.46
0.45
0.46
0.43
0.46
0.38
0.46
0.41
0.46
0.14
0.46
0.11
0.46
0.37
0.46
0.38
0.46
0.39
0.46
0.09
0.46
0.45
0.46
0.45
0.46
0.33
0.46
0.44
0.46
0.44
0.46
0.4
0.46
0.37
0.46
0.1
0.46
0.34
0.46
0.45
0.46
0.21
0.46
0.1
0.46
0.35
0.46
0.42
0.46
0.42
0.46
0.39
0.46
0.11
0.46
0.43
0.46
0.1
0.46
0.19
0.46
0.38
0.46
0.38
0.46
0.37
0.46
0.37
0.46
0.38
0.46
0.42
0.46
0.09
0.46
0.37
0.46
0.13


0.24
0.46
0.09
0.46
0.11
0.46
0.09
0.46
0.43
0.46
0.44
0.46
0.22
0.46
0.44
0.46
0.36
0.46
0.16
0.46
0.1
0.46
0.36
0.46
0.42
0.46
0.24
0.46
0.34
0.46
0.36
0.46
0.44
0.46
0.29
0.46
0.4
0.46
0.21
0.46
0.4
0.46
0.41
0.46
0.3
0.46
0.21
0.46
0.16
0.46
0.42
0.46
0.1
0.46
0.1
0.46
0.42
0.46
0.11
0.46
0.22
0.46
0.37
0.46
0.43
0.46
0.42
0.46
0.09
0.46
0.45
0.46
0.46
0.46
0.19
0.46
0.41
0.46
0.31
0.46
0.3
0.46
0.34
0.46
0.09
0.46
0.1
0.46
0.4
0.46
0.34
0.46
0.45
0.46
0.44
0.46
0.37
0.46
0.09
0.46
0.09
0.46
0.16
0.46
0.45
0.46
0.36
0.46
0.38
0.46
0.1
0.46
0.45
0.46
0.18
0.46
0.1
0.46
0.11
0.46
0.26
0.46
0.42
0.46
0.2
0.46
0.43
0.46
0.16
0.46
0.1
0.46
0.39
0.46
0.43
0.46
0.4
0.46
0.3
0.46
0.09
0.46
0.29
0.46
0.43
0.46
0.11
0.46
0.45
0.46
0.4
0.46
0.32
0.46
0.11
0.46
0.43
0.46
0.4
0.46
0.38
0.46
0.4
0.46
0.1
0.46
0.1
0.46
0.41
0.46
0.2
0.46
0.22
0.46
0.1
0.46
0.09
0.46
0.4
0.46
0.36
0.46
0.45
0.46
0.34
0.46
0.11
0.46
0.43
0.46
0.43
0.46
0.43
0.46
0.2
0.46
0.11
0.46
0.14
0.46
0.37
0.46
0.46
0.46
0.11

{'satisfaction_level': 0.39103704121623334, 'last_evaluation': 0.43770583321299344, 'number_project': 0.37329731806942673, 'average_montly_hours': 0.4393500967745508, 'time_spend_company': 0.39248009137933887}
BestAttribute isnumber_project
BestAttributeType isnumerical
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 2 of level 1 number of examples of this node 2501
Type4
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0

2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
length of left children is 1042
length of right children is 1042
{'Work_accident': [0, 1], 'promotion_last_5years': [0, 1], 'sales': ['IT', 'RandD', 'accounting', 'hr', 'management', 'marketing', 'product_mng', 'sales', 'support', 'technical'], 'salary': ['high', 'low', 'medium']}
calculating Impurity
{'Work_accident': 0.20642077111329604, 'promotion_last_5years': 0.2132729336873575, 'sales': 0.21231260337314992, 'salary': 0.21151899655583267}
{'satisfaction_level': [0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16,

...

{'satisfaction_level': 0.13845198428337693, 'last_evaluation': 0.1100608664518439, 'number_project': 0.11753239307418871, 'average_montly_hours': 0.13815789473684212, 'time_spend_company': 0.12681212311769396}
BestAttribute islast_evaluation
BestAttributeType isnumerical
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.99 of level 4 number of examples of this node 152
Type4
0.94
0.99
0.99
0.99
0.95
0.99
0.83
0.99
0.83
0.99
0.83
0.99
0.97
0.99
0.98
0.99
0.85
0.99
0.89
0.99
0.84
0.99
0.86
0.99
0.99
0.99
0.93
0.99
0.87
0.99
0.85
0.99
0.98
0.99
0.95
0.99
0.9
0.99
0.83
0.99
0.94
0.99
0.82
0.99
0.94
0.99
0.84
0.99
0.87
0.99
0.86
0.99
0.95
0.99
0.89
0.99
0.88
0.99
0.88
0.99
0.98
0.99
0.88
0.99
0.86
0.99
0.82
0.99
0.93
0.99
0.82
0.99
0.82
0.99
0.97
0.99
0.85
0.99
0.99
0.99
0.88
0.99
0.86
0.99
0.91
0.99
0.94
0.99
0.96
0.99
0.98
0.99
0.85
0.99
0.96
0.99
0.98
0.99
0.87
0.99
0.85
0.99
0.85
0.99
0.84
0.99
0.95
0.99
0.9
0.99
0.95
0.99
0.85
0.99
0.9

{'Work_accident': 0.20584720443875373, 'promotion_last_5years': 0.20255172241262853, 'sales': 0.20041440057017668, 'salary': 0.19616533701040742}
{'satisfaction_level': [0.48, 0.49, 0.5, 0.51, 0.52, 0.56, 0.57, 0.58, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.68, 0.69, 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.95, 0.96, 0.98], 'last_evaluation': [0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0], 'number_project': [2, 3, 4, 5, 6], 'average_montly_hours': [217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 279, 286, 287, 307], 'time_spend_company': [5, 6, 7, 8, 10]}
calculating Impu

{'satisfaction_level': 0.059961315280464215, 'last_evaluation': 0.07763686902508561, 'number_project': 0.045144914794229406, 'average_montly_hours': 0.07182664410058026, 'time_spend_company': 0.07620529929613558}
BestAttribute isnumber_project
BestAttributeType isnumerical
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 3 of level 6 number of examples of this node 517
Type4
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
2.0
3.0
3.0
3.0
2.0
3.0
2.0
3.0
length of left children is 11
length of right children is 11
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 7 number of examples of this node 11
Type3
{'Work_accident': [0, 1], 'promotion_last_5years': [0, 1], 'sales': ['IT', 'RandD', 'accounting', 'hr', 'management', 'marketing', 'product_mng', 'sales', 'support', 'technical'], 'salary': ['high', 'low', 'medium']}
calculating Impurity
{'Work_accident': 0.04227865059067

0.88
0.92
0.72
0.92
0.8
0.92
0.9
0.92
0.87
0.92
0.73
0.92
0.87
0.92
0.83
0.92
0.87
0.92
0.77
0.92
0.84
0.92
0.92
0.92
0.92
0.92
0.81
0.92
0.72
0.92
0.77
0.92
0.79
0.92
0.84
0.92
0.87
0.92
0.76
0.92
0.9
0.92
0.74
0.92
0.86
0.92
0.87
0.92
0.76
0.92
0.91
0.92
0.86
0.92
0.82
0.92
0.84
0.92
0.85
0.92
0.75
0.92
0.76
0.92
0.85
0.92
0.83
0.92
0.81
0.92
0.79
0.92
0.75
0.92
0.77
0.92
0.88
0.92
0.86
0.92
0.83
0.92
0.9
0.92
0.92
0.92
0.79
0.92
0.91
0.92
0.82
0.92
0.78
0.92
0.75
0.92
0.85
0.92
0.74
0.92
0.78
0.92
0.72
0.92
0.8
0.92
0.89
0.92
0.82
0.92
0.85
0.92
0.83
0.92
0.8
0.92
0.73
0.92
0.76
0.92
0.85
0.92
0.84
0.92
0.76
0.92
0.9
0.92
0.89
0.92
0.87
0.92
0.79
0.92
0.79
0.92
0.92
0.92
0.84
0.92
0.84
0.92
0.9
0.92
0.74
0.92
0.83
0.92
0.88
0.92
0.88
0.92
0.84
0.92
0.78
0.92
0.84
0.92
0.91
0.92
0.84
0.92
0.91
0.92
0.79
0.92
0.78
0.92
0.88
0.92
0.79
0.92
0.8
0.92
0.88
0.92
0.81
0.92
0.82
0.92
0.77
0.92
0.83
0.92
0.8
0.92
0.87
0.92
0.9
0.92
0.76
0.92
0.79
0.92
0.77
0.92
0.87
0.92
0.74
0.92
0.77
0.92
0

{'satisfaction_level': 0.023486968883272785, 'last_evaluation': 0.023653119037983464, 'number_project': 0.023694397283531413, 'average_montly_hours': 0.023496012759170654, 'time_spend_company': 0.02370874020494274}
BestAttribute isWork_accident
BestAttributeType iscategorical
Decision attribute type is categoricaldecisionAttribute is Work_accident decisionAttributeValue is 0 of level 10 number of examples of this node 500
Type4
length of left children is 470
length of right children is 470
{'Work_accident': [0], 'promotion_last_5years': [0], 'sales': ['IT', 'RandD', 'accounting', 'hr', 'management', 'marketing', 'product_mng', 'sales', 'support', 'technical'], 'salary': ['high', 'low', 'medium']}
calculating Impurity
{'Work_accident': 999, 'promotion_last_5years': 999, 'sales': 0.0167832946823701, 'salary': 0.016865197307466235}
{'satisfaction_level': [0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92], 'last_eval

In [35]:
def RecursiveFunctionToPredict(i,r,TestSet):
    if r.decisionAttribute == 'leafNode':
        if r.decisionAttributeValue == 'Positive':
            return '1'
        else:
            #print("I am here in leaft")
            return '0'
    else:
        if r.decisionAttributeType == 'categorical': 
            
            AttrValue = str(TestSet.at[int(i),str(r.decisionAttribute)])
            
            if AttrValue == str(r.decisionAttributeValue):
                return RecursiveFunctionToPredict(i,r.leftChild,TestSet)
            else:
                return RecursiveFunctionToPredict(i,r.rightChild,TestSet)
            
        elif r.decisionAttributeType == 'numerical':
            
            AttrValue = float(str(TestSet.at[int(i),str(r.decisionAttribute)]))
            
            if AttrValue <= float(str(r.decisionAttributeValue)):
                return RecursiveFunctionToPredict(i,r.leftChild,TestSet)
            else:
                return RecursiveFunctionToPredict(i,r.rightChild,TestSet)
            

In [ ]:
float(str(0.51))

In [94]:
    TrainCopy = train.copy()
    GivenTestData = TrainCopy
    #GivenTestData = simpletest
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    for i in list(GivenTestData.index.values):
        if str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(1):
            TP = TP + 1
        elif str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(0):
            FN = FN + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(1):
            FP = FP + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(0):
            TN = TN + 1
    print("Results On Training Data")
    print("Using Misclassification Rate as impurity measure")
    print("-------------------------------------------------")
    print("True Positive are    "+ str(TP))
    print("True Negatives are   "+ str(TN))
    print("False Positive are   "+ str(FP))
    print("False Negatives are  "+str(FN))
    
    Precision = (TP/(TP+FP))
    Recall = (TP/(TP+FN))
    F1_Score = 2*((1/Recall)+(1/Precision))
    accuracy = (TN+TP)/(TN+TP+FP+FN)
    print("Precsion is          "+str(Precision))
    print("Recall is            "+str(Recall))
    print("F1_Score is          "+str(F1_Score))
    print("Accuracy is          "+str(accuracy))
    print("-------------------------------------------------")
    print("( Here 1's  are positive 0's are negative)")
    len(val)

Results On Training Data
Using Misclassification Rate as impurity measure
-------------------------------------------------
True Positive are    1972
True Negatives are   6819
False Positive are   17
False Negatives are  181
Precsion is          0.9914529914529915
Recall is            0.9159312587087785
F1_Score is          4.200811359026369
Accuracy is          0.9779730782066971
-------------------------------------------------
( Here 1's  are positive 0's are negative)


2249

In [15]:
temp1 = [1,2,3]
temp2 = temp1.copy()
temp2[1] = 6
temp1

[1, 2, 3]

In [34]:
def RightChild(j):
    if j.rightChild != null:
    print(j.rightChild)
    return RightChild(j.rightChild)

    

print(RootNode)
print(RootNode.rightChild)
print(RootNode.rightChild.rightChild)
print(RootNode.rightChild.rightChild.rightChild)
print(RootNode.rightChild.rightChild.rightChild.rightChild)

Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.46 of level 0 number of examples of this node 8989
Decision attribute type is numericaldecisionAttribute is time_spend_company decisionAttributeValue is 4 of level 1 number of examples of this node 6488
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.8 of level 2 number of examples of this node 1186
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 216 of level 3 number of examples of this node 738
Decision attribute type is numericaldecisionAttribute is time_spend_company decisionAttributeValue is 6 of level 4 number of examples of this node 582


In [ ]:
def predict(RootNode,NameOfTestFile):
    GivenTestData = pd.read_csv(str(NameOfTestFile))
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    return GivenTestData
Prediction = predict(RootNode,'sample_test.csv')
Prediction.head()   

In [42]:
GivenTestData.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,predict
1,0.89,0.93,4,249,3,0,0,0,sales,low,0
12,0.20,0.68,5,167,7,0,0,0,marketing,medium,0
13,0.68,0.55,6,181,3,0,0,0,IT,medium,0
14,0.41,0.63,2,145,2,0,0,0,product_mng,low,0
16,0.93,0.87,3,154,3,1,0,0,IT,low,0


In [52]:
train.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.10,0.90,7,286,4,0,1,0,sales,low
2,0.38,0.50,2,132,3,0,1,0,accounting,low
3,0.95,0.71,4,151,4,0,0,0,sales,medium
4,0.84,0.84,5,163,3,0,0,0,technical,low
5,0.76,0.94,4,168,6,0,0,0,technical,medium


In [53]:
train.to_csv('UsedTrainingData.csv')


In [54]:
len(train)

8989

In [55]:
val.to_csv('UsedValidationData.csv')

In [56]:
len(val)

2249

In [72]:
simpletest = pd.read_csv('UsedTrainingData.csv')

In [68]:
len(simpletest)

8989

In [69]:
simpletest.head()


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.10,0.90,7,286,4,0,1,0,sales,low
1,0.38,0.50,2,132,3,0,1,0,accounting,low
2,0.95,0.71,4,151,4,0,0,0,sales,medium
3,0.84,0.84,5,163,3,0,0,0,technical,low
4,0.76,0.94,4,168,6,0,0,0,technical,medium
